# Project: Search and Sample Return

**The goals / steps of this project are the following:**  

**Training / Calibration**  

* Download the simulator and take data in "Training Mode" - DONE
* Test out the functions in the Jupyter Notebook provided - DONE / submitted with project
* Add functions to detect obstacles and samples of interest (golden rocks) - DONE and successful
* Fill in the `process_image()` function with the appropriate image processing steps (perspective transform, color threshold etc.) to get from raw images to a map.  The `output_image` you create in this step should demonstrate that your mapping pipeline works. DONE and successful to greater than 40% coverage and approximately 80% fidelity.
* Use `moviepy` to process the images in your saved dataset with the `process_image()` function.  Include the video you produce as part of your submission - DONE as part of notebook stage though screencap used to create submission video.

**Autonomous Navigation / Mapping**

* Fill in the `perception_step()` function within the `perception.py` script with the appropriate image processing functions to create a map and update `Rover()` data (similar to what you did with `process_image()` in the notebook). - DONE
* Fill in the `decision_step()` function within the `decision.py` script with conditional statements that take into consideration the outputs of the `perception_step()` in deciding how to issue throttle, brake and steering commands - DONE 
* Iterate on your perception and decision function until your rover does a reasonable (need to define metric) job of navigating and mapping - DONE according to baseline submission criteria (40% coverage, greater than 60% fidelity)

[//]: # (Image References)

[image2]: ../calibration_images/example_grid1.jpg
[image3]: ../calibration_images/example_rock2.jpg


[Rubric](https://review.udacity.com/#!/rubrics/916/view) Points

# Writeup / README
Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  
---

##### Notebook Analysis
#### 1. Run the functions provided in the notebook on test images (first with the test data provided, next on data you have recorded). Add/modify functions to allow for color selection of obstacles and rock samples.

In the Jupyter notebook I used at first the images provided in the repo, and then later inserted some of my own. Below is an example of the calibration image I used for setting the source transform area and an image of a rock that I took from generated output images from the simulator.

![Grid on: Area used for mapping transformed pixels][image2]


![Image of one of the sample rocks][image3]

Here I experimented with both the methods given in the classes (using the numpy array and setting the threshold limits) and also using openCV "simple thresholding" and "weighted thresholding" methods. I found that at least on static images the openCV simple threshold method seemed to produce visibly better data, with less overlap between actual and perceived navigable terrain, but in the simulation runs there was little difference. The weighted image was not suitable for "zeroing out" navigable vs non-navigable terrain and was rejected. For these reasons I opted to stay with the class methods in the final project. 

For the provided class methods, the relevant lines of code are:

def color_thresh(img, rgb_thresh=(160, 160, 160)):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    # Require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True"
    # where threshold was met
    above_thresh = (img[:,:,0] > rgb_thresh[0]) \
               & (img[:,:,1] > rgb_thresh[1]) \
               & (img[:,:,2] > rgb_thresh[2]) \
            #data.pitch  
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

The above is for finding navigable terrain, with lighter pixels above the threshold. For obstacles, we take the pixels BELOW the threshold.

And for opencv, simple thresholding:

In [1]:
def color_thresh(img):
    # Create an array of zeros same xy size as img, but single channel
    color_select = np.zeros_like(img[:,:,0])
    As the threshold function can only use single channel grayscale, separate the source image into 3 channels
    a1 = np.array(img)
    a = a1[:,:,0]
    b1 = np.array(img)
    b = b1[:,:,1]
    c1 = np.array(img)
    c = c1[:,:,2]

    #run the cv2 threshold code on each channel. I used the TRUNC flag as this produced a finer edge to the navigable terrain
    ret,trunc_a = cv2.threshold(a,155,255,cv2.THRESH_TRUNC)
    ret,trunc_b = cv2.threshold(b,155,255,cv2.THRESH_TRUNC)
    ret,trunc_c = cv2.threshold(c,155,255,cv2.THRESH_TRUNC)
   
    # As before, require that each pixel be above all three threshold values in RGB
    # above_thresh will now contain a boolean array with "True" where threshold was met
    
    above_thresh = (trunc_a[:,:] == 155) \
                & (trunc_b[:,:] == 155) \
                & (trunc_c[:,:] == 155)
    
    # Index the array of zeros with the boolean array and set to 1
    color_select[above_thresh] = 1
    # Return the binary image
    return color_select

SyntaxError: invalid syntax (<ipython-input-1-fe2b27869d6f>, line 4)

BELOW: Output images from the TRUNC warp and thresh function, followed by comparison of the two methods, overlaid onto the original, non-thresholded image. The opencv method (middle) gives slightly finer interpetation of the area of "blown sand" up the cavern walls than the classroom method (bottom)

[//]:

[image4]: ../writeup_images/ThreshedWarpedCV2TRUNC.jpg
[image5]: ../writeup_images/opencvBLEND.jpg
[image6]: ../writeup_images/classBLEND.jpg
[image7]: ../writeup_images/4.jpg
    
![Warped / thresholded][image4]

![OpenCV][image5]

![Classroom method][image6]

#### 2. Populate the `process_image()` function with the appropriate analysis steps to map pixels identifying navigable terrain, obstacles and rock samples into a worldmap.  Run `process_image()` on your test data using the `moviepy` functions provided to create video output of your result.

Below are the supporting functions which are then called by the process_image() function, listed underneath. These are mostly "stock" from the classroom lessons with some tweaks.

In [ ]:
# Define a function to perform a perspective transform
# I've used the example grid image above to choose source points for the
# grid cell in front of the rover (each grid cell is 1 square meter in the sim)

def perspect_transform(img, src, dst):           
    #define the transformation matrix required by the function
    M = cv2.getPerspectiveTransform(src, dst)
    #apply the function
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))# keep same size as input image
    #create a mask area so that the region outside the rover's camera is not factored into the final map image
    mask = cv2.warpPerspective(np.ones_like(img[:,:,0]), M, (img.shape[1], img.shape[0]))
    return warped, mask


# Define calibration box in source (actual) and destination (desired) coordinates
# These source and destination points are defined to warp the image
# to a grid where each 10x10 pixel square represents 1 square meter
# The destination box will be 2*dst_size on each side
dst_size = 5 
# Set a bottom offset to account for the fact that the bottom of the image 
# is not the position of the rover but a bit in front of it

bottom_offset = 6
source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
destination = np.float32([[image.shape[1]/2 - dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - 2*dst_size - bottom_offset], 
                  [image.shape[1]/2 - dst_size, image.shape[0] - 2*dst_size - bottom_offset],
                  ])
#finally call the function and provide it with the needed arguments:
warped, mask = perspect_transform(grid_img, source, destination)


# Define a function to convert from image coordinates to rover coordinates
def rover_coords(binary_img):
    # Identify nonzero pixels
    ypos, xpos = binary_img.nonzero()
    # Calculate pixel positions with reference to the rover position being at the 
    # center bottom of the image.  
    x_pixel = -(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[1]/2 ).astype(np.float)
    return x_pixel, y_pixel

# Define a function to convert to radial coords in rover space
def to_polar_coords(x_pixel, y_pixel):
    # Convert (x_pixel, y_pixel) to (distance, angle) 
    # in polar coordinates in rover space
    # Calculate distance to each pixel
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    # Calculate angle away from vertical for each pixel
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

# Define a function to map rover space pixels to world space
def rotate_pix(xpix, ypix, yaw):
    # Convert yaw to radians
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
                            
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated

def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated


# Define a function to apply rotation and translation (and clipping)
# Once you define the two functions above this function should work
def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    # Apply rotation
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    # Apply translation
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    # Perform rotation, translation and clipping all at once
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    # Return the result
    return x_pix_world, y_pix_world

Below is an image of the outputs from the above functions. Clockwise from top left: Rover camera image, warped camera image, warped and thresholded image, and the transformed-to-rover coords image plus average steer angle

[//]:
    
[image7]: ../writeup_images/4.jpg
    
![ALT TEXT][image7]

In [ ]:
#Note that the arrow seen in the last image is generated by the following code:
arrow_length = 100
x_arrow = arrow_length * np.cos(mean_dir)
y_arrow = arrow_length * np.sin(mean_dir)
plt.arrow(0, 0, x_arrow, y_arrow, color='red', zorder=2, head_width=10, width=2)

#similar code is used later in order to define the steer angle for the rover

Now defining the process_image function, which will call the above functions and pass the data to the sim

In [ ]:
# Define a function to pass stored images to,
# reading rover position and yaw angle from csv file
# This function will be used by moviepy to create an output video
def process_image(img):
    #In the notebook, the instance of the class is called data, however in the functioning rover it is called "Rover".
   
    # The aim of this function is to: 
    # 1) Define source and destination points for perspective transform and then apply the transform.
    # 2) Apply perspective transform
    # 3) Apply color threshold to identify navigable terrain/obstacles (rock samples are handled separately below)
    # 4) Convert thresholded image pixel values to rover-centric coords
    # 5) Convert rover-centric pixel values to world coords
    # 6) Update worldmap (to be displayed on right side of screen)
        # Example: data.worldmap[obstacle_y_world, obstacle_x_world, 0] += 1
        #          data.worldmap[rock_y_world, rock_x_world, 1] += 1
        #          data.worldmap[navigable_y_world, navigable_x_world, 2] += 1
        
        
    # 1) Define source and destination points for perspective transform and then apply the transform. 
    # Here we are calling the transform function defined above:
    warped, mask = perspect_transform(img, source, destination)
    #Apply colour theshold to identify navigable terrin / obstacles
    threshed = color_thresh(warped)
    #create the obstacle map
    obs_map = np.absolute(np.float32(threshed) - 1) * mask
    xpix, ypix = rover_coords(threshed)
    #convert rover-centric pixel values to world co-ordinates
    world_size = data.worldmap.shape[0]
    #define the size of the target area 
    scale = 2 * dst_size
    #retrieve data from the data bucket
    xpos = data.xpos[data.count]
    ypos = data.ypos[data.count]
    yaw = data.yaw[data.count]
    pitch_value = data.pitch[data.count]
    
    #Go ahead and create the maps to be overlaid:
    x_world, y_world = pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale)
    obsxpix, obsypix = rover_coords(obs_map) 
    obs_x_world, obs_y_world = pix_to_world(obsxpix, obsypix, xpos, ypos, yaw, world_size, scale)
        
    #This section is designed to only update the worldmap when the pitch angle is within certain limits. This is because
    #the transform function only produces valid results for a flat plane. In the autononmous driving portion I also applied
    #a limit for roll.
    
    #Navigable terrain
    if pitch_value < 0.01:
        data.worldmap[y_world, x_world, 2] = 255
    elif pitch_value > 355:
        data.worldmap[y_world, x_world, 2] = 255
    else: 
        pass
    
    #Obstacles
    if pitch_value < 0.01:
        data.worldmap[obs_y_world, obs_x_world, 0] = 255
    elif pitch_value > 355:
        data.worldmap[obs_y_world, obs_x_world, 0] = 255
    else:
        pass
    
    #Lets make our map pixels the right colours:
    nav_pix = data.worldmap[:, :, 2] > 0
    data.worldmap[nav_pix, 0] = 0
          
    #Threshold function for finding rocks
    rock_map = find_rocks(warped, levels=(110, 110, 50))
    if rock_map.any():
        rock_x, rock_y = rover_coords(rock_map)
        rock_x_world, rock_y_world = pix_to_world(rock_x, rock_y, xpos, ypos, yaw, world_size, scale)
        data.worldmap[rock_y_world, rock_x_world, :] = 255
    
    # 7) Make a mosaic image, this is the stick code from the notebook
        # First create a blank image (can be whatever shape you like)
    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1]*2, 3))
        # Next you can populate regions of the image with various output
        # Here I'm putting the original image in the upper left hand corner
    output_image[0:img.shape[0], 0:img.shape[1]] = img

        # Let's create more images to add to the mosaic, first a warped image
    warped, mask = perspect_transform(img, source, destination)
        # Add the warped image in the upper right hand corner
    output_image[0:img.shape[0], img.shape[1]:] = warped

        # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.5, 0)
        # Flip map overlay so y-axis points upward and add to output_image 
    output_image[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)
    
    #defining some variables to allow us to, for example, view our pitch and position:
    pitch_text = str(data.pitch[data.count])
    xpos_text = str(data.xpos[data.count])
    ypos_text = str(data.ypos[data.count])

        # Then putting some text over the image
    cv2.putText(output_image, pitch_text,  (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(output_image, xpos_text,  (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    
    return output_image

### Autonomous Navigation and Mapping

#### 1. Fill in the `perception_step()` (at the bottom of the `perception.py` script) and `decision_step()` (in `decision.py`) functions in the autonomous mapping scripts.

The perception step was filled consistent with the explanations provided above, and once the rover_angles were being output correctly, the rover now had some basic ability to self drive.

I used the standard "average angle" method for steering, which seemed to work very well. I would have liked to have tried a wall following approach, however due to time constraints I went with the average angle.

#### Decision Step, and Getting Stuck

Resolution: 1920 x 1080
Graphics Settings: Fantastic
FPS: 45

I experimented with trying to implement a "stuck" mode for the rover. To do this I tried a number of things, including creating a list inside the rover object that updated perception that kept the absolute value of the product of the global coordinates (See code snippet below). This list, at my sims 45FPS, was capable of containing around 10 seconds of data, and so if the first and last indexes were pretty much the same, then the rover was stuck (see code below that I ultimately removed from the decision step).

I also tried comparing speed vs throttle, the logic being that if the throttle was open but the rover not moving, then this would mean it was stuck. This would have helped with the instances where the camera view "clipped" through the rocks and led the rover to think that there was clear space in front of it.

One problem I ran into was that the rover could sometimes not differentiate between reasons why velocity was zero, such as "just started the sim" vs "I have hit something". I tried implementing a count so that the stuck routine would not begin until 1000 time steps had elapsed, allowing the rover to begin moving.

Another problem I ran into was having other if statements compete with one another, which would lead to "unsticking" procedures like backing up or turning being interrupted or just oscillating on the spot.

Unfortunately I could not make any of these approaches work consistently, I believe due to some confusion with the if statement topography. I would like to learn more about implementing decision trees in python. However as my rover could fairly regularly complete the required 40% mapping and had good (80%) fidelity, I decided this was sufficient given the time spent on the project.

I plan to revisit this project in the future and try some alternative approaches, as well as bolster my knowledge on implementing decision trees in code.

AUTONOMOUS DRIVING PERFORMANCE

As noted above, I was not able to implement an unsticking routine. The mapping process can therefore fail if the rover drives into a rock as its first action. Clearly an unsticking procedure would help here.

In terms of navigation, the rover does a reasonably good job, and it will eventually find the average angle that leads to it mapping the entire area given a stuck-free run.

In [ ]:
# code to implement average angle steering. It takes a mean of the angles to each of the pixels in frame.
Rover.steer = np.clip(np.mean(Rover.nav_angles * 180/np.pi), -15, 15)

In [ ]:
#code to create buffer of formatted last 10 seconds position data
	Rover.xypos_mult = abs((Rover.pos[0] * Rover.pos[1]))
	#this populates the empty list from beginning to end, and then shifts the data to the right once
	#the array size is 450. At 45fps this equates to 10 seconds of data.
	if (len(Rover.buffer_xypos) < 450):
		Rover.buffer_xypos.insert(0, Rover.xypos_mult)
	elif (len(Rover.buffer_xypos) == 450):
		Rover.buffer_xypos.pop()
		Rover.buffer_xypos.insert(0, Rover.xypos_mult)
	image = Rover.img

In [ ]:
#statement briefly added to the decision step to try and implement the 'stuck' routine. It checks if the rover has not moved
# for 10 seconds (data comes from Rover.buffer_xypos), whether the rover can see ahead (obstacles such as rocks are either
#transparent due to clipping or some navigable terrain is visible). Finally it checks to make sure the sim has not just
#started up. This is because I was struggling to make the logic differentiate between "I have just started" and "I am stuck"
elif Rover.vel == 0 and (abs(Rover.buffer_xypos[-1]) - (Rover.buffer_xypos[0]) < 2) and (len(Rover.nav_angles) >= Rover.stop_forward) and Rover.count > 1000:
    Throttle = 0
    Rover.steer = 15
    Rover.mode = 'Stuck'